In [1]:
# imports
import sys
import os
import socket
import pandas as pd
import glob

In [2]:
# function to format dates to datetime and replace nans in eventdate with sysdate
def formatdate(data):
    data['sysdate'] = data['sysdate'].str.replace('/', '')
    data['sysdate']= pd.to_datetime(data['sysdate'], format='%d%m%Y')
    data['eventdate']= data['eventdate'].str.replace('/', '')
    data['eventdate'] = pd.to_datetime(data['eventdate'], format='%d%m%Y')
    data['eventdate'].fillna(data['sysdate'], inplace = True)
    return data

In [3]:
os.getcwd()

'/mnt/maths/tws21'

In [4]:
print(glob.glob("/mnt/*"))

['/mnt/cm-shared', '/mnt/epp', '/mnt/nfs3', '/mnt/pact', '/mnt/lustre', '/mnt/nfs2', '/mnt/shared', '/mnt/ebinstall', '/mnt/maths', '/mnt/amo', '/mnt/geog', '/mnt/astro-xcs', '/mnt/export-xcs', '/mnt/xmm-xcs', '/mnt/sim-xcs']


In [5]:
next(os.walk('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730'))

('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730',
 ['bsms2730'],
 ['exmaple.txt', 'example.txt'])

In [6]:
next(os.walk('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730'))

('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730',
 ['Dictionary Browsers',
  'Marco Antipsychotics Files',
  'Lookups',
  'Linked Data',
  'Primary Care',
  'Documentation'],
 ['19_050.zip'])

In [7]:
next(os.walk('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD'))

('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD',
 [],
 ['19_050_gold_extract_therapy_072.txt',
  '19_050_gold_extract_test_010.txt',
  '19_050_gold_extract_therapy_055.txt',
  '19_050_gold_extract_therapy_113.txt',
  '19_050_gold_extract_therapy_033.txt',
  '19_050_gold_extract_therapy_077.txt',
  '19_050_gold_extract_therapy_023.txt',
  '19_050_gold_extract_clinical_028.txt',
  '19_050_gold_extract_test_018.txt',
  '19_050_gold_extract_test_027.txt',
  '19_050_gold_extract_clinical_010.txt',
  '19_050_gold_extract_test_012.txt',
  '19_050_gold_extract_additional_001.txt',
  '19_050_gold_extract_test_023.txt',
  '19_050_gold_extract_therapy_036.txt',
  '19_050_gold_extract_therapy_062.txt',
  '19_050_gold_extract_clinical_024.txt',
  '19_050_gold_extract_therapy_007.txt',
  '19_050_gold_extract_therapy_079.txt',
  '19_050_gold_extract_therapy_060.txt',
  '19_050_gold_extract_therapy_099.txt',
  '19_050_gold_extract_therapy_061.txt',
  '19_050_gold_extr

In [8]:
# read in dementia codes
dementia_codes = pd.read_excel('/mnt/maths/tws21/dementia_codes.xlsx')
dementia_codes.head()

,medcode,readcode,readterm,type,probable,possible,diagnosis,history,admin,assessment,suspected,referral,drugs
0,1917,F110.00,Alzheimer's disease,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,7664,Eu00.00,[X]Dementia in Alzheimer's disease,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,8195,Eu00z11,[X]Alzheimer's dementia unspec,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,11379,Eu00112,"[X]Senile dementia,Alzheimer's type",Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,16797,F110000,Alzheimer's disease with early onset,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# select only diagnosis codes
dementia_med_codes = dementia_codes[dementia_codes['diagnosis'] == 1.0]['medcode']
dementia_med_codes

0       1917
1       7664
2       8195
3      11379
4      16797
       ...  
136    49513
137    53446
138    55222
139    55838
153    30706
Name: medcode, Length: 94, dtype: int64

In [10]:
path = '/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD/'
clinical_files = glob.glob(path + "*clinical*.txt") # list all files in the directory with "clinical" in filename

#read  all the clinical files in and add to a list
clinical_list = []
for filename in clinical_files:    
    df = pd.read_csv(filename, sep="\t")
    df = df[df['medcode'].isin(list(dementia_med_codes))]
    df = formatdate(df)
    clinical_list.append(df)

In [11]:
#merge all the clinical files
clinical_merged = pd.concat(clinical_list, axis=0)
clinical_merged.head()

,e_patid,eventdate,sysdate,constype,consid,medcode,episode,enttype,adid
408,10010191,2000-01-24,2004-02-10,3,136438,4500,4,2,0
423,10010191,2001-09-03,2001-10-04,3,136455,1917,4,2,0
5893,100349220,1995-11-28,2016-04-04,3,1483382,1916,4,2,0
5908,100349220,2001-02-27,2016-04-04,3,1483418,8634,1,2,0
5911,100349220,1998-07-10,2016-04-04,3,1483382,8634,4,2,0


In [12]:
# output all dementia diagnosis rows from clinical files
clinical_merged.to_parquet(os.getcwd() + '/dementia_patients_data.parquet')